In [72]:
from os.path import abspath, join, dirname
import pandas as pd
import sqlite3
import re

curr_dir = abspath('2.manejo.ipynb')
conn = sqlite3.connect(join(dirname(curr_dir), 'db', 'db.sqlite'))

df = pd.read_sql_query('SELECT * FROM `filtered`', conn)
df.head()

,title,author,year,abstract,doi,keywords,journal,month,publisher,address,...,funding-text,oa,organization,book-group-author,author-keywords,abbrev-source-title,publication-stage,source,correspondence-address,coden
0,Big Data Information Security Risk Framework D...,"Yuxiang, Li and Haitao, Zhang and Jingyi, Bo a...",2021,This paper proposes the big data information s...,10.1109/I-SMAC52330.2021.9640740,Big Data Information; DDos Attack; Framework D...,None,None,IEEE,"345 E 47TH ST, NEW YORK, NY 10017 USA",...,"In 2018, the Ministry of Education's Industry-...",None,IEEE; SCAD Inst Technol,IEEE,None,None,None,None,None,None
1,Iterative Dichotomiser Posteriori Method Based...,"Dhiyanesh, B. and Karthick, K. and Radha, R. a...",2023,Cloud computing (CC) is an advanced technology...,10.32604/csse.2023.024691,ID3 (Iterative dichotomiser 3) maximum multifa...,COMPUTER SYSTEMS SCIENCE AND ENGINEERING,None,TECH SCIENCE PRESS,"871 CORONADO CENTER DR, SUTE 200, HENDERSON, N...",...,None,hybrid,None,None,None,None,None,None,None,None
2,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Cloud computing is supported by Fog computing ...,10.1007/s11277-023-10407-2,Deep Reinforcement Learning (DRL) model; Fog-a...,WIRELESS PERSONAL COMMUNICATIONS,June,SPRINGER,"ONE NEW YORK PLAZA, SUITE 4600, NEW YORK, NY, ...",...,None,None,None,None,None,None,None,None,None,None
3,DDoS Detection Systems for Cloud Data Storage,"Cao, Ting and Mao, Jianzhou and Bhattacharya, ...",2021,Distributed Denial-of-Service (DDoS) attack is...,10.1109/TPSISA52974.2021.00021,cloud; application layer DDoS attack; anomaly ...,None,None,IEEE COMPUTER SOC,"10662 LOS VAQUEROS CIRCLE, PO BOX 3014, LOS AL...",...,This work is supported in part by the U.S. Nat...,None,IEEE; IEEE Comp Soc,IEEE Comp Soc,None,None,None,None,None,None
4,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,"In the recent years, security is the major con...",10.1007/s12065-019-00244-3,Cloud computing; Network; Traffic; DDoS; Packe...,EVOLUTIONARY INTELLIGENCE,June,SPRINGER HEIDELBERG,"TIERGARTENSTRASSE 17, D-69121 HEIDELBERG, GERMANY",...,None,None,None,None,None,None,None,None,None,None


In [73]:
df = df[df['type'].str.contains('article', case=False)]
df.shape

(194, 54)

In [74]:
rel_cols = [
    'title',
    'author',
    'year',
    'abstract',
    'keywords',
    'doi',
    'url',
    'publisher',
    'type',
    'language'
]

def make_pucv_url(url):
    if pd.isna(url):
        return None
    pattern = r'https?://([a-zA-Z0-9.-]+)/.*'
    match = re.match(pattern, url)
    if match:
        domain = match.group(1)
        domain_gr = domain.replace('.', '-')
        return url.replace(domain, f'{domain_gr}.pucv.idm.oclc.org')

df = df.drop(columns=[col for col in df.columns if col not in rel_cols])
df['doi'] = df['doi'].apply(lambda x: x.replace('\\', ''))
df['doi_url'] = df['doi'].apply(lambda x: f'https://doi.org/{x}')
df['pucv_url'] = df['url'].apply(make_pucv_url)

df.shape

(194, 12)

In [75]:
temas: dict[str, dict[str, any]] = {
    'metaheuristics': {
        'keywords': ['metaheuristic', 'metaheuristics'],
        'dois': []
    },
    'machine_learning': {
        'keywords': ['machine learning', 'deep learning'],
        'dois': []
    },
    'ddos': {
        'keywords': ['ddos', 'denial of service', 'ddos attack'],
        'dois': []
    },
    'iot': {
        'keywords': ['iot', 'internet of things'],
        'dois': []
    },
    'cloud': {
        'keywords': ['cloud', 'cloud computing', 'cloud architecture'],
        'dois': []
    },
    'cybersecurity': {
        'keywords': ['cybersecurity', 'vulnerability', 'security'],
        'dois': []
    },
}

out_dir = join(dirname(curr_dir), 'out')

def contiene_palabra_clave_en_keywords(texto, palabras_clave):
    if pd.isna(texto):
        return False
    # Separar los keywords por ';' y eliminar espacios en blanco
    keywords_list = [kw.strip() for kw in texto.split(';')]
    for keyword in keywords_list:
        for palabra in palabras_clave:
            if palabra.lower() in keyword.lower():
                return True
    return False


def contiene_palabra_clave_en_texto(texto, palabras_clave):
    if pd.isna(texto):
        return False
    for palabra in palabras_clave:
        if palabra.lower() in texto.lower():
            return True
    return False


def get_filtered_df(t):
    return df[
        df['keywords']
            .apply(lambda x: contiene_palabra_clave_en_keywords(x,temas[t]['keywords'])) 
        |
        df['title']
            .apply(lambda x: contiene_palabra_clave_en_texto(x,temas[t]['keywords']))
        ]


for k, v in temas.items():
    print(k)
    filtered = get_filtered_df(k)
    v['dois'] = filtered['doi'].tolist()
    print(filtered.shape)

metaheuristics
(10, 12)
machine_learning
(98, 12)
ddos
(125, 12)
iot
(49, 12)
cloud
(132, 12)
cybersecurity
(99, 12)


In [76]:
df['categories'] = df \
    .apply(lambda x: ', '.join(str(t) for t in [k for k, v in temas.items() 
                                                if x['doi'] in v['dois']]), 
           axis=1)
df.head()

,title,author,year,abstract,doi,keywords,publisher,type,language,url,doi_url,pucv_url,categories
1,Iterative Dichotomiser Posteriori Method Based...,"Dhiyanesh, B. and Karthick, K. and Radha, R. a...",2023,Cloud computing (CC) is an advanced technology...,10.32604/csse.2023.024691,ID3 (Iterative dichotomiser 3) maximum multifa...,TECH SCIENCE PRESS,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.32604/csse.2023.024691,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud"
2,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Cloud computing is supported by Fog computing ...,10.1007/s11277-023-10407-2,Deep Reinforcement Learning (DRL) model; Fog-a...,SPRINGER,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.1007/s11277-023-10407-2,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud"
4,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,"In the recent years, security is the major con...",10.1007/s12065-019-00244-3,Cloud computing; Network; Traffic; DDoS; Packe...,SPRINGER HEIDELBERG,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.1007/s12065-019-00244-3,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud"
6,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Cloud computing has revolutionized the informa...,10.3390/s24051418,cloud computing; cybersecurity risks; DDoS; ma...,MDPI,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.3390/s24051418,https://www-webofscience-com.pucv.idm.oclc.org...,"machine_learning, ddos, cloud, cybersecurity"
7,An SDN-Assisted Defense Mechduanism for the Sh...,"Agrawal, Neha and Tapaswi, Shashikala",2021,The integration of cloud computing with Softwa...,10.1007/s10922-020-09580-7,Cloud computing security; Software defined net...,SPRINGER,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.1007/s10922-020-09580-7,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud, cybersecurity"


In [77]:
df.to_csv(join(out_dir, 'references.csv'), index=False, sep=';')

In [78]:
df.to_sql('references', conn, if_exists='append', index=False)

194

In [2]:
from os.path import abspath, join, dirname
import pandas as pd
import sqlite3
import re
new_csv_path = '/Users/sgarciad/Downloads'

selected = pd.read_csv(join(new_csv_path, 'list.csv'), sep=';', encoding='utf-8')
selected.head()

,corr,doi
0,2,10.1007/s11277-023-10407-2
1,3,10.1007/s12065-019-00244-3
2,4,10.3390/s24051418
3,9,10.1016/j.comnet.2022.109138
4,30,10.1109/ACCESS.2021.3126834


In [3]:
typeform = pd.read_csv(join(new_csv_path, 'typeform-export.csv'))
typeform.head()

,title,TL;DR,Summarized Abstract,Summarized Introduction,Objectives,Literature Survey,Problem Statement,Dataset,Methods Used,Population Sample,Results,Limitations,Conclusions,Metaheuristics,ML/DL Algorithm
0,art191.pdf,- The paper introduces the BNT-CBPOADL techniq...,- The research paper focuses on introducing a ...,- The research paper focuses on detecting botn...,- The research objective of the paper is to in...,- Wei et al. proposed a lightweight NIDS for b...,- The problem statement discussed in the paper...,- The study utilized the Bot-IoT database for ...,"- Z-score normalization for pre-processing, CB...",- The population sample size in the study was ...,- The BNT-CBPOADL technique achieved an averag...,- The research paper does not explicitly menti...,- The study introduces a Botnet Detection usin...,- The metaheuristic optimization approach used...,- The study utilized the Chaotic Binary Pelica...
1,art188.pdf,- The research paper presents a Hybrid Metaheu...,- The research paper focuses on Botnet detecti...,- The research paper focuses on botnet detecti...,- The research objectives of the paper include...,- Vinayakumar et al. presented a botnet detect...,- The research paper addresses the issue of bo...,- The study utilized the N-BaIoT dataset for s...,- The paper presents a Hybrid Metaheuristics w...,- The population sample size used in the study...,- The HMMLB-BND method achieved an average acc...,- The research paper does not explicitly menti...,- The study presented a Hybrid Metaheuristics ...,- The study mentions the use of Hybrid Metaheu...,- The study mentions the utilization of a hybr...
2,art182.pdf,- The paper proposes a novel framework for DDo...,- The research paper focuses on detecting DDoS...,- The research paper focuses on detecting Dist...,- The research objectives of the paper include...,- Aneetha et al. proposed updated self-organiz...,- The research paper addresses the problem of ...,- NSL-KDD dataset was used for experiments and...,- The research paper utilizes Deep Belief Netw...,- The research paper does not provide specific...,- The research paper proposed a DDoS attack de...,- The limitations mentioned in the paper inclu...,- The paper proposed a DDoS attack detection m...,- The research paper does not mention the use ...,- The ML/DL algorithms used in the study are L...
3,art168.pdf,- The paper presents an efficient DoS attack d...,- The paper presents an efficient DoS attack d...,- The research paper focuses on the critical i...,- The research objectives of the paper include...,- The literature review of the paper provides ...,- The research paper addresses the critical is...,- The study utilized the KDD CUP 99 dataset as...,- The paper utilizes the Oppositional Crow Sea...,- The population sample size in the proposed D...,- The proposed DoS attack detection system out...,The limitations mentioned in the paper are not...,- The paper proposes an attack detection techn...,- The metaheuristic algorithms used in the res...,- The study mentions the use of the Opposition...
4,art164.pdf,- The research paper proposes a Hybrid Algorit...,- The research paper focuses on detecting and ...,- The research paper focuses on detecting and ...,- The research objectives of the paper include...,- The literature review of the paper focuses o...,- The research paper addresses the problem of ...,- The study utilized the KDD CUP 99 dataset fo...,- The paper utilizes a Hybrid algorithm-based ...,- The research paper does not provide specific...,- The paper presents a Hybrid algorithm-based ...,- The limitations mentioned in the paper inclu...,- The paper presents a Hybrid algorithm-based ...,- The research paper does not mention the use ...,- The study does not specify the use of any ma...


In [6]:
selected['file'] = selected['corr'].apply(lambda x: f'art{x}.pdf')
selected.head()

,corr,doi,file
0,2,10.1007/s11277-023-10407-2,art2.pdf
1,3,10.1007/s12065-019-00244-3,art3.pdf
2,4,10.3390/s24051418,art4.pdf
3,9,10.1016/j.comnet.2022.109138,art9.pdf
4,30,10.1109/ACCESS.2021.3126834,art30.pdf


In [8]:
both = pd.merge(selected, typeform, left_on='file', right_on='title', 
                how='inner')
both.shape

(36, 18)

In [9]:
both

,corr,doi,file,title,TL;DR,Summarized Abstract,Summarized Introduction,Objectives,Literature Survey,Problem Statement,Dataset,Methods Used,Population Sample,Results,Limitations,Conclusions,Metaheuristics,ML/DL Algorithm
0,2,10.1007/s11277-023-10407-2,art2.pdf,art2.pdf,- The paper presents a Deep Reinforcement Lear...,- The research paper focuses on a Deep Reinfor...,"- Cloud computing faces security challenges, w...",- The research objectives of the paper include...,- A Multi-Layer Perceptron (MLP)-based DDoS at...,- The research paper addresses the issue of Di...,- The datasets used in the study include the H...,- Deep Reinforcement Learning (DRL) and Long S...,- The research paper does not provide specific...,- The proposed RDL-LSTM-2.2 scheme in the pape...,- The paper does not explicitly mention any li...,- The paper proposes a Deep Reinforcement Lear...,- The research paper does not mention the use ...,- The study utilizes Deep Reinforcement Learni...
1,3,10.1007/s12065-019-00244-3,art3.pdf,art3.pdf,- The paper proposes a behaviour-based DDoS de...,- The paper focuses on addressing security con...,- The paper focuses on addressing security con...,- The research objectives of the paper are to ...,- Various research studies have been conducted...,- The research paper addresses the major secur...,- The study utilized a customized network envi...,- The paper proposed a behaviour-based DDoS de...,- The research paper does not provide specific...,- The proposed behaviour-based DDoS detection ...,- The proposed model cannot identify both shor...,- The paper proposed a behavior-based DDoS det...,- The research paper does not mention the use ...,- The research paper does not specify any mach...
2,4,10.3390/s24051418,art4.pdf,art4.pdf,- The paper introduces a novel machine learnin...,- The research paper presents a novel machine ...,"- Cloud computing has transformed IT, with 95%...",- The research objectives of the paper include...,- The literature review of the paper compares ...,- The research paper addresses the problem of ...,"- The study utilized the CICDDoS2019 dataset, ...",- The paper utilizes a Bayesian-based Convolut...,- The population sample size in the study invo...,- The paper presents the performance of the Ba...,- Prior studies have been constrained by vario...,- The research paper introduces two innovative...,- Metaheuristics are not specified in the rese...,- The study applied the Bayesian-based Convolu...
3,9,10.1016/j.comnet.2022.109138,art9.pdf,art9.pdf,- The paper proposes an efficient SVM-based DE...,- The paper proposes an efficient SVM-based DE...,- The research paper focuses on detecting Dist...,- The research objectives of the paper involve...,- Kushwah et al. proposed a voting extreme lea...,- The research paper addresses the rising thre...,"- NSL-KDD database with 20,000 training sample...",- The paper proposes an approach that involves...,- The population sample size used in the study...,- The paper presents experimental analyses for...,- The limitations mentioned in the paper inclu...,- The paper proposed a detection attack system...,- The research paper does mention the use of m...,- The study mentions the use of the Support Ve...
4,30,10.1109/ACCESS.2021.3126834,art30.pdf,art30.pdf,- The paper explores the application of machin...,- The paper discusses the challenges of anomal...,"- The research paper titled ""Machine Learning ...",- The research objectives of the paper include...,- Machine Learning (ML) is a powerful tool for...,- The research paper focuses on the challenges...,- Hosseini and Azizi used public datasets to d...,- The paper discusses the implementation of su...,The research paper does not provide specific i...,- The paper presents a comprehensive survey on...,- Challenges in anomaly detection under differ...,- Machine learning is proving itself in anomal...,- Metaheuristics are not specifically mentione...,- The study primarily focuses on Machine Learn...
5,32,10.3390/electronics120

In [10]:
drop_cols = [
    'title',
    'corr'
]
both.drop(columns=drop_cols, inplace=True)


In [11]:
both

,doi,file,TL;DR,Summarized Abstract,Summarized Introduction,Objectives,Literature Survey,Problem Statement,Dataset,Methods Used,Population Sample,Results,Limitations,Conclusions,Metaheuristics,ML/DL Algorithm
0,10.1007/s11277-023-10407-2,art2.pdf,- The paper presents a Deep Reinforcement Lear...,- The research paper focuses on a Deep Reinfor...,"- Cloud computing faces security challenges, w...",- The research objectives of the paper include...,- A Multi-Layer Perceptron (MLP)-based DDoS at...,- The research paper addresses the issue of Di...,- The datasets used in the study include the H...,- Deep Reinforcement Learning (DRL) and Long S...,- The research paper does not provide specific...,- The proposed RDL-LSTM-2.2 scheme in the pape...,- The paper does not explicitly mention any li...,- The paper proposes a Deep Reinforcement Lear...,- The research paper does not mention the use ...,- The study utilizes Deep Reinforcement Learni...
1,10.1007/s12065-019-00244-3,art3.pdf,- The paper proposes a behaviour-based DDoS de...,- The paper focuses on addressing security con...,- The paper focuses on addressing security con...,- The research objectives of the paper are to ...,- Various research studies have been conducted...,- The research paper addresses the major secur...,- The study utilized a customized network envi...,- The paper proposed a behaviour-based DDoS de...,- The research paper does not provide specific...,- The proposed behaviour-based DDoS detection ...,- The proposed model cannot identify both shor...,- The paper proposed a behavior-based DDoS det...,- The research paper does not mention the use ...,- The research paper does not specify any mach...
2,10.3390/s24051418,art4.pdf,- The paper introduces a novel machine learnin...,- The research paper presents a novel machine ...,"- Cloud computing has transformed IT, with 95%...",- The research objectives of the paper include...,- The literature review of the paper compares ...,- The research paper addresses the problem of ...,"- The study utilized the CICDDoS2019 dataset, ...",- The paper utilizes a Bayesian-based Convolut...,- The population sample size in the study invo...,- The paper presents the performance of the Ba...,- Prior studies have been constrained by vario...,- The research paper introduces two innovative...,- Metaheuristics are not specified in the rese...,- The study applied the Bayesian-based Convolu...
3,10.1016/j.comnet.2022.109138,art9.pdf,- The paper proposes an efficient SVM-based DE...,- The paper proposes an efficient SVM-based DE...,- The research paper focuses on detecting Dist...,- The research objectives of the paper involve...,- Kushwah et al. proposed a voting extreme lea...,- The research paper addresses the rising thre...,"- NSL-KDD database with 20,000 training sample...",- The paper proposes an approach that involves...,- The population sample size used in the study...,- The paper presents experimental analyses for...,- The limitations mentioned in the paper inclu...,- The paper proposed a detection attack system...,- The research paper does mention the use of m...,- The study mentions the use of the Support Ve...
4,10.1109/ACCESS.2021.3126834,art30.pdf,- The paper explores the application of machin...,- The paper discusses the challenges of anomal...,"- The research paper titled ""Machine Learning ...",- The research objectives of the paper include...,- Machine Learning (ML) is a powerful tool for...,- The research paper focuses on the challenges...,- Hosseini and Azizi used public datasets to d...,- The paper discusses the implementation of su...,The research paper does not provide specific i...,- The paper presents a comprehensive survey on...,- Challenges in anomaly detection under differ...,- Machine learning is proving itself in anomal...,- Metaheuristics are not specifically mentione...,- The study primarily focuses on Machine Learn...
5,10.3390/electronics12061423,art32.pdf,- The paper focuses on enhancing intrusion det...,- The

In [12]:
curr_dir = abspath('2.manejo.ipynb')
conn = sqlite3.connect(join(dirname(curr_dir), 'db', 'db.sqlite'))

both.to_sql('processed', conn, if_exists='replace', index=False)

36

In [20]:
info = pd.read_sql_query('SELECT * FROM `references`', conn)
info

,title,author,year,abstract,doi,keywords,publisher,type,language,url,doi_url,pucv_url,categories
0,Iterative Dichotomiser Posteriori Method Based...,"Dhiyanesh, B. and Karthick, K. and Radha, R. a...",2023,Cloud computing (CC) is an advanced technology...,10.32604/csse.2023.024691,ID3 (Iterative dichotomiser 3) maximum multifa...,TECH SCIENCE PRESS,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.32604/csse.2023.024691,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud"
1,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Cloud computing is supported by Fog computing ...,10.1007/s11277-023-10407-2,Deep Reinforcement Learning (DRL) model; Fog-a...,SPRINGER,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.1007/s11277-023-10407-2,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud"
2,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,"In the recent years, security is the major con...",10.1007/s12065-019-00244-3,Cloud computing; Network; Traffic; DDoS; Packe...,SPRINGER HEIDELBERG,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.1007/s12065-019-00244-3,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud"
3,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Cloud computing has revolutionized the informa...,10.3390/s24051418,cloud computing; cybersecurity risks; DDoS; ma...,MDPI,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.3390/s24051418,https://www-webofscience-com.pucv.idm.oclc.org...,"machine_learning, ddos, cloud, cybersecurity"
4,An SDN-Assisted Defense Mechduanism for the Sh...,"Agrawal, Neha and Tapaswi, Shashikala",2021,The integration of cloud computing with Softwa...,10.1007/s10922-020-09580-7,Cloud computing security; Software defined net...,SPRINGER,Article,English,https://www.webofscience.com/wos/woscc/full-re...,https://doi.org/10.1007/s10922-020-09580-7,https://www-webofscience-com.pucv.idm.oclc.org...,"ddos, cloud, cybersecurity"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,A novel dynamic framework to detect DDoS in SD...,"Shakil, Muhammad and Fuad Yousif Mohammed, Ala...",2022,Security is a crucial factor in the continuous...,10.1002/ett.3622,Clustering algorithms; Network security; Distr...,John Wiley and Sons Inc,Article,English,https://www.scopus.com/inward/record.uri?eid=2...,https://doi.org/10.1002/ett.3622,https://www-scopus-com.pucv.idm.oclc.org/inwar...,"metaheuristics, machine_learning, ddos, cybers..."
190,Modeling of Botnet Detection Using Chaotic Bin...,"Alrowais, Fadwa and Eltahir, Majdy M. and Alja...",2023,"Nowadays, there are ample amounts of Internet ...",10.1109/ACCESS.2023.3332690,Botnet; Cybersecurity; Deep learning; Feature ...,Institute of Electrical and Electronics Engine...,Article,English,https://www.scopus.com/inward/record.uri?eid=2...,https://doi.org/10.1109/ACCESS.2023.3332690,https://www-scopus-com.pucv.idm.oclc.org/inwar...,"metaheuristics, machine_learning, iot, cyberse..."
191,Vehicular Network Intrusion Detection Using a ...,"Manderna, Ankit and Kumar, Sushil and Dohare, ...",2023,Vehicle malfunctions have a direct impact on b...,10.3390/s23218772,algorithm; article; artificial intelligence; a...,None,Article,English,https://www.scopus.com/inward/record.uri?eid=2...,https://doi.org/10.3390/s23218772,https://www-scopus-com.pucv.idm.oclc.org/inwar...,"metaheuristics, machine_learning, cybersecurity"
192,Detecting attacks on IoT devices with probabil...,"Toğaçar, Mesut",2022,Technologies that enable physical objects to c...,10.1002/ett.4418,Bayesian networks; Deep learning; Internet of ...,John Wiley and Sons Inc,Article,English,https://www.scopus.com/inward/record.uri?eid=2...,https://doi.org/10.1002/ett.4418,https://www-scopus-com.p

In [21]:
info_drop_cols = ['language', 'doi_url', 'url', 'categories']
info.drop(columns=info_drop_cols, inplace=True)
info['keywords'] = info['keywords'].apply(lambda x: (x.replace('; ', ';'))
                                          .lower())
info

,title,author,year,abstract,doi,keywords,publisher,type,pucv_url
0,Iterative Dichotomiser Posteriori Method Based...,"Dhiyanesh, B. and Karthick, K. and Radha, R. a...",2023,Cloud computing (CC) is an advanced technology...,10.32604/csse.2023.024691,id3 (iterative dichotomiser 3) maximum multifa...,TECH SCIENCE PRESS,Article,https://www-webofscience-com.pucv.idm.oclc.org...
1,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Cloud computing is supported by Fog computing ...,10.1007/s11277-023-10407-2,deep reinforcement learning (drl) model;fog-as...,SPRINGER,Article,https://www-webofscience-com.pucv.idm.oclc.org...
2,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,"In the recent years, security is the major con...",10.1007/s12065-019-00244-3,cloud computing;network;traffic;ddos;packet an...,SPRINGER HEIDELBERG,Article,https://www-webofscience-com.pucv.idm.oclc.org...
3,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Cloud computing has revolutionized the informa...,10.3390/s24051418,cloud computing;cybersecurity risks;ddos;machi...,MDPI,Article,https://www-webofscience-com.pucv.idm.oclc.org...
4,An SDN-Assisted Defense Mechduanism for the Sh...,"Agrawal, Neha and Tapaswi, Shashikala",2021,The integration of cloud computing with Softwa...,10.1007/s10922-020-09580-7,cloud computing security;software defined netw...,SPRINGER,Article,https://www-webofscience-com.pucv.idm.oclc.org...
...,...,...,...,...,...,...,...,...,...
189,A novel dynamic framework to detect DDoS in SD...,"Shakil, Muhammad and Fuad Yousif Mohammed, Ala...",2022,Security is a crucial factor in the continuous...,10.1002/ett.3622,clustering algorithms;network security;distrib...,John Wiley and Sons Inc,Article,https://www-scopus-com.pucv.idm.oclc.org/inwar...
190,Modeling of Botnet Detection Using Chaotic Bin...,"Alrowais, Fadwa and Eltahir, Majdy M. and Alja...",2023,"Nowadays, there are ample amounts of Internet ...",10.1109/ACCESS.2023.3332690,botnet;cybersecurity;deep learning;feature ext...,Institute of Electrical and Electronics Engine...,Article,https://www-scopus-com.pucv.idm.oclc.org/inwar...
191,Vehicular Network Intrusion Detection Using a ...,"Manderna, Ankit and Kumar, Sushil and Dohare, ...",2023,Vehicle malfunctions have a direct impact on b...,10.3390/s23218772,algorithm;article;artificial intelligence;assa...,None,Article,https://www-scopus-com.pucv.idm.oclc.org/inwar...
192,Detecting attacks on IoT devices with probabil...,"Toğaçar, Mesut",2022,Technologies that enable physical objects to c...,10.1002/ett.4418,bayesian networks;deep learning;internet of th...,John Wiley and Sons Inc,Article,https://www-scopus-com.pucv.idm.oclc.org/inwar...


In [22]:
all = pd.merge(info, both, left_on='doi', right_on='doi', how='inner')
all.shape

(36, 24)

In [23]:
all

,title,author,year,abstract,doi,keywords,publisher,type,pucv_url,file,...,Literature Survey,Problem Statement,Dataset,Methods Used,Population Sample,Results,Limitations,Conclusions,Metaheuristics,ML/DL Algorithm
0,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Cloud computing is supported by Fog computing ...,10.1007/s11277-023-10407-2,deep reinforcement learning (drl) model;fog-as...,SPRINGER,Article,https://www-webofscience-com.pucv.idm.oclc.org...,art2.pdf,...,- A Multi-Layer Perceptron (MLP)-based DDoS at...,- The research paper addresses the issue of Di...,- The datasets used in the study include the H...,- Deep Reinforcement Learning (DRL) and Long S...,- The research paper does not provide specific...,- The proposed RDL-LSTM-2.2 scheme in the pape...,- The paper does not explicitly mention any li...,- The paper proposes a Deep Reinforcement Lear...,- The research paper does not mention the use ...,- The study utilizes Deep Reinforcement Learni...
1,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,"In the recent years, security is the major con...",10.1007/s12065-019-00244-3,cloud computing;network;traffic;ddos;packet an...,SPRINGER HEIDELBERG,Article,https://www-webofscience-com.pucv.idm.oclc.org...,art3.pdf,...,- Various research studies have been conducted...,- The research paper addresses the major secur...,- The study utilized a customized network envi...,- The paper proposed a behaviour-based DDoS de...,- The research paper does not provide specific...,- The proposed behaviour-based DDoS detection ...,- The proposed model cannot identify both shor...,- The paper proposed a behavior-based DDoS det...,- The research paper does not mention the use ...,- The research paper does not specify any mach...
2,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Cloud computing has revolutionized the informa...,10.3390/s24051418,cloud computing;cybersecurity risks;ddos;machi...,MDPI,Article,https://www-webofscience-com.pucv.idm.oclc.org...,art4.pdf,...,- The literature review of the paper compares ...,- The research paper addresses the problem of ...,"- The study utilized the CICDDoS2019 dataset, ...",- The paper utilizes a Bayesian-based Convolut...,- The population sample size in the study invo...,- The paper presents the performance of the Ba...,- Prior studies have been constrained by vario...,- The research paper introduces two innovative...,- Metaheuristics are not specified in the rese...,- The study applied the Bayesian-based Convolu...
3,An efficient SVM based DEHO classifier to dete...,"Alam, Gowthul M. M. and Kumar, Jerald Nirmal S...",2022,Distributed denial of service (DDoS) attacks i...,10.1016/j.comnet.2022.109138,ddos attack;cloud;deho;svm;kpca;databases;data...,ELSEVIER,Article,https://www-webofscience-com.pucv.idm.oclc.org...,art9.pdf,...,- Kushwah et al. proposed a voting extreme lea...,- The research paper addresses the rising thre...,"- NSL-KDD database with 20,000 training sample...",- The paper proposes an approach that involves...,- The population sample size used in the study...,- The paper presents experimental analyses for...,- The limitations mentioned in the paper inclu...,- The paper proposed a detection attack system...,- The research paper does mention the use of m...,- The study mentions the use of the Support Ve...
4,Machine Learning in Network Anomaly Detection:...,"Wang, Song and Balarezo, Juan Fernando and Kan...",2021,Anomalies could be the threats to the network ...,10.1109/ACCESS.2021.3126834,anomaly detection;training;machine learning;pr...,IEEE-INST ELECTRICAL ELECTRONICS ENGINEERS INC,Article,https://www-webofscience-com.pucv.idm.oclc.org...,art30.pdf,...,- Machine Learning (ML) is a powerful tool for...,- The research paper focuses on the challenges...,- Hosseini and Azizi used public datasets to d...,- The paper discusse

In [24]:
cols_ordered = [
    'doi',
    'file',
    'title',
    'author',
    'year',
    'type',
    'publisher',
    'keywords',
    'pucv_url',
    'abstract',
    'TL;DR',
    'Summarized Abstract',
    'Summarized Introduction',
    'Objectives',
    'Literature Survey',
    'Problem Statement',
    'Dataset',
    'Methods Used',
    'Population Sample',
    'Results',
    'Limitations',
    'Conclusions',
    'Metaheuristics',
    'ML/DL Algorithm'
]

all = all[cols_ordered]
all

,doi,file,title,author,year,type,publisher,keywords,pucv_url,abstract,...,Literature Survey,Problem Statement,Dataset,Methods Used,Population Sample,Results,Limitations,Conclusions,Metaheuristics,ML/DL Algorithm
0,10.1007/s11277-023-10407-2,art2.pdf,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Article,SPRINGER,deep reinforcement learning (drl) model;fog-as...,https://www-webofscience-com.pucv.idm.oclc.org...,Cloud computing is supported by Fog computing ...,...,- A Multi-Layer Perceptron (MLP)-based DDoS at...,- The research paper addresses the issue of Di...,- The datasets used in the study include the H...,- Deep Reinforcement Learning (DRL) and Long S...,- The research paper does not provide specific...,- The proposed RDL-LSTM-2.2 scheme in the pape...,- The paper does not explicitly mention any li...,- The paper proposes a Deep Reinforcement Lear...,- The research paper does not mention the use ...,- The study utilizes Deep Reinforcement Learni...
1,10.1007/s12065-019-00244-3,art3.pdf,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,Article,SPRINGER HEIDELBERG,cloud computing;network;traffic;ddos;packet an...,https://www-webofscience-com.pucv.idm.oclc.org...,"In the recent years, security is the major con...",...,- Various research studies have been conducted...,- The research paper addresses the major secur...,- The study utilized a customized network envi...,- The paper proposed a behaviour-based DDoS de...,- The research paper does not provide specific...,- The proposed behaviour-based DDoS detection ...,- The proposed model cannot identify both shor...,- The paper proposed a behavior-based DDoS det...,- The research paper does not mention the use ...,- The research paper does not specify any mach...
2,10.3390/s24051418,art4.pdf,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Article,MDPI,cloud computing;cybersecurity risks;ddos;machi...,https://www-webofscience-com.pucv.idm.oclc.org...,Cloud computing has revolutionized the informa...,...,- The literature review of the paper compares ...,- The research paper addresses the problem of ...,"- The study utilized the CICDDoS2019 dataset, ...",- The paper utilizes a Bayesian-based Convolut...,- The population sample size in the study invo...,- The paper presents the performance of the Ba...,- Prior studies have been constrained by vario...,- The research paper introduces two innovative...,- Metaheuristics are not specified in the rese...,- The study applied the Bayesian-based Convolu...
3,10.1016/j.comnet.2022.109138,art9.pdf,An efficient SVM based DEHO classifier to dete...,"Alam, Gowthul M. M. and Kumar, Jerald Nirmal S...",2022,Article,ELSEVIER,ddos attack;cloud;deho;svm;kpca;databases;data...,https://www-webofscience-com.pucv.idm.oclc.org...,Distributed denial of service (DDoS) attacks i...,...,- Kushwah et al. proposed a voting extreme lea...,- The research paper addresses the rising thre...,"- NSL-KDD database with 20,000 training sample...",- The paper proposes an approach that involves...,- The population sample size used in the study...,- The paper presents experimental analyses for...,- The limitations mentioned in the paper inclu...,- The paper proposed a detection attack system...,- The research paper does mention the use of m...,- The study mentions the use of the Support Ve...
4,10.1109/ACCESS.2021.3126834,art30.pdf,Machine Learning in Network Anomaly Detection:...,"Wang, Song and Balarezo, Juan Fernando and Kan...",2021,Article,IEEE-INST ELECTRICAL ELECTRONICS ENGINEERS INC,anomaly detection;training;machine learning;pr...,https://www-webofscience-com.pucv.idm.oclc.org...,Anomalies could be the threats to the network ...,...,- Machine Learning (ML) is a powerful tool for...,- The research paper focuses on the challenges...,- Hosseini and Azizi used public datasets to d...,- The paper discusse

In [25]:
new_cols = {
    'doi': 'DOI',
    'file': 'Nombre archivo',
    'title': 'Título',
    'author': 'Autor/es',
    'year': 'Año',
    'type': 'Tipo',
    'publisher': 'Editorial',
    'keywords': 'Palabras clave',
    'pucv_url': 'URL',
    'abstract': 'Abstract',
    'TL;DR': 'TL;DR',
    'Summarized Abstract': 'Resumen abstract',
    'Summarized Introduction': 'Resumen introducción',
    'Objectives': 'Objetivos',
    'Literature Survey': 'Revisión literatura',   
    'Problem Statement': 'Problema planteado',
    'Dataset': 'Dataset',
    'Methods Used': 'Métodos utilizados',
    'Population Sample': 'Muestra de la población',
    'Results': 'Resumen Resultados',
    'Limitations': 'Limitaciones',
    'Conclusions': 'Conclusiones',
    'Metaheuristics': 'Metaheurística/s utilizada/s',
    'ML/DL Algorithm': 'Algoritmo/s ML/DL utilizado/s'
}

all.rename(columns=new_cols, inplace=True)

In [26]:
all

,DOI,Nombre archivo,Título,Autor/es,Año,Tipo,Editorial,Palabras clave,URL,Abstract,...,Revisión literatura,Problema planteado,Dataset,Métodos utilizados,Muestra de la población,Resumen Resultados,Limitaciones,Conclusiones,Metaheurística/s utilizada/s,Algoritmo/s ML/DL utilizado/s
0,10.1007/s11277-023-10407-2,art2.pdf,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Article,SPRINGER,deep reinforcement learning (drl) model;fog-as...,https://www-webofscience-com.pucv.idm.oclc.org...,Cloud computing is supported by Fog computing ...,...,- A Multi-Layer Perceptron (MLP)-based DDoS at...,- The research paper addresses the issue of Di...,- The datasets used in the study include the H...,- Deep Reinforcement Learning (DRL) and Long S...,- The research paper does not provide specific...,- The proposed RDL-LSTM-2.2 scheme in the pape...,- The paper does not explicitly mention any li...,- The paper proposes a Deep Reinforcement Lear...,- The research paper does not mention the use ...,- The study utilizes Deep Reinforcement Learni...
1,10.1007/s12065-019-00244-3,art3.pdf,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,Article,SPRINGER HEIDELBERG,cloud computing;network;traffic;ddos;packet an...,https://www-webofscience-com.pucv.idm.oclc.org...,"In the recent years, security is the major con...",...,- Various research studies have been conducted...,- The research paper addresses the major secur...,- The study utilized a customized network envi...,- The paper proposed a behaviour-based DDoS de...,- The research paper does not provide specific...,- The proposed behaviour-based DDoS detection ...,- The proposed model cannot identify both shor...,- The paper proposed a behavior-based DDoS det...,- The research paper does not mention the use ...,- The research paper does not specify any mach...
2,10.3390/s24051418,art4.pdf,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Article,MDPI,cloud computing;cybersecurity risks;ddos;machi...,https://www-webofscience-com.pucv.idm.oclc.org...,Cloud computing has revolutionized the informa...,...,- The literature review of the paper compares ...,- The research paper addresses the problem of ...,"- The study utilized the CICDDoS2019 dataset, ...",- The paper utilizes a Bayesian-based Convolut...,- The population sample size in the study invo...,- The paper presents the performance of the Ba...,- Prior studies have been constrained by vario...,- The research paper introduces two innovative...,- Metaheuristics are not specified in the rese...,- The study applied the Bayesian-based Convolu...
3,10.1016/j.comnet.2022.109138,art9.pdf,An efficient SVM based DEHO classifier to dete...,"Alam, Gowthul M. M. and Kumar, Jerald Nirmal S...",2022,Article,ELSEVIER,ddos attack;cloud;deho;svm;kpca;databases;data...,https://www-webofscience-com.pucv.idm.oclc.org...,Distributed denial of service (DDoS) attacks i...,...,- Kushwah et al. proposed a voting extreme lea...,- The research paper addresses the rising thre...,"- NSL-KDD database with 20,000 training sample...",- The paper proposes an approach that involves...,- The population sample size used in the study...,- The paper presents experimental analyses for...,- The limitations mentioned in the paper inclu...,- The paper proposed a detection attack system...,- The research paper does mention the use of m...,- The study mentions the use of the Support Ve...
4,10.1109/ACCESS.2021.3126834,art30.pdf,Machine Learning in Network Anomaly Detection:...,"Wang, Song and Balarezo, Juan Fernando and Kan...",2021,Article,IEEE-INST ELECTRICAL ELECTRONICS ENGINEERS INC,anomaly detection;training;machine learning;pr...,https://www-webofscience-com.pucv.idm.oclc.org...,Anomalies could be the threats to the network ...,...,- Machine Learning (ML) is a powerful tool for...,- The research paper focuses on the challenges...,-

In [29]:
all.to_csv(join(new_csv_path, 'final.csv'), index=False, sep=';')

In [31]:
all.to_excel(join(new_csv_path, 'final.xlsx'), index=False)

In [32]:
all.to_sql('final', conn, if_exists='replace', index=False)

36

In [34]:
all.to_csv(join(new_csv_path, 'final.csv'), index=False)

In [4]:
import sqlite3
import pandas as pd
from os.path import abspath, join, dirname

curr_dir = abspath('2.manejo.ipynb')
conn = sqlite3.connect(join(dirname(curr_dir), 'db', 'db.sqlite'))

df = pd.read_sql_query('SELECT * FROM `final`', conn)

In [5]:
df

,DOI,Nombre archivo,Título,Autor/es,Año,Tipo,Editorial,Palabras clave,URL,Abstract,...,Revisión literatura,Problema planteado,Dataset,Métodos utilizados,Muestra de la población,Resumen Resultados,Limitaciones,Conclusiones,Metaheurística/s utilizada/s,Algoritmo/s ML/DL utilizado/s
0,10.1007/s11277-023-10407-2,art2.pdf,A Deep Reinforcement Learning-based DDoS Attac...,"Janakiraman, Sengathir and Priya, M. Deva",2023,Article,SPRINGER,deep reinforcement learning (drl) model;fog-as...,https://www-webofscience-com.pucv.idm.oclc.org...,Cloud computing is supported by Fog computing ...,...,- A Multi-Layer Perceptron (MLP)-based DDoS at...,- The research paper addresses the issue of Di...,- The datasets used in the study include the H...,- Deep Reinforcement Learning (DRL) and Long S...,- The research paper does not provide specific...,- The proposed RDL-LSTM-2.2 scheme in the pape...,- The paper does not explicitly mention any li...,- The paper proposes a Deep Reinforcement Lear...,- The research paper does not mention the use ...,- The study utilizes Deep Reinforcement Learni...
1,10.1007/s12065-019-00244-3,art3.pdf,Design and analysis of behaviour based DDoS de...,"Penukonda, Qubeb Shaik Mohammed and Paramasiva...",2021,Article,SPRINGER HEIDELBERG,cloud computing;network;traffic;ddos;packet an...,https://www-webofscience-com.pucv.idm.oclc.org...,"In the recent years, security is the major con...",...,- Various research studies have been conducted...,- The research paper addresses the major secur...,- The study utilized a customized network envi...,- The paper proposed a behaviour-based DDoS de...,- The research paper does not provide specific...,- The proposed behaviour-based DDoS detection ...,- The proposed model cannot identify both shor...,- The paper proposed a behavior-based DDoS det...,- The research paper does not mention the use ...,- The research paper does not specify any mach...
2,10.3390/s24051418,art4.pdf,Novel Machine Learning Approach for DDoS Cloud...,"Alsaleh, Ibtihal and Al-Samawi, Aida and Nissi...",2024,Article,MDPI,cloud computing;cybersecurity risks;ddos;machi...,https://www-webofscience-com.pucv.idm.oclc.org...,Cloud computing has revolutionized the informa...,...,- The literature review of the paper compares ...,- The research paper addresses the problem of ...,"- The study utilized the CICDDoS2019 dataset, ...",- The paper utilizes a Bayesian-based Convolut...,- The population sample size in the study invo...,- The paper presents the performance of the Ba...,- Prior studies have been constrained by vario...,- The research paper introduces two innovative...,- Metaheuristics are not specified in the rese...,- The study applied the Bayesian-based Convolu...
3,10.1016/j.comnet.2022.109138,art9.pdf,An efficient SVM based DEHO classifier to dete...,"Alam, Gowthul M. M. and Kumar, Jerald Nirmal S...",2022,Article,ELSEVIER,ddos attack;cloud;deho;svm;kpca;databases;data...,https://www-webofscience-com.pucv.idm.oclc.org...,Distributed denial of service (DDoS) attacks i...,...,- Kushwah et al. proposed a voting extreme lea...,- The research paper addresses the rising thre...,"- NSL-KDD database with 20,000 training sample...",- The paper proposes an approach that involves...,- The population sample size used in the study...,- The paper presents experimental analyses for...,- The limitations mentioned in the paper inclu...,- The paper proposed a detection attack system...,- The research paper does mention the use of m...,- The study mentions the use of the Support Ve...
4,10.1109/ACCESS.2021.3126834,art30.pdf,Machine Learning in Network Anomaly Detection:...,"Wang, Song and Balarezo, Juan Fernando and Kan...",2021,Article,IEEE-INST ELECTRICAL ELECTRONICS ENGINEERS INC,anomaly detection;training;machine learning;pr...,https://www-webofscience-com.pucv.idm.oclc.org...,Anomalies could be the threats to the network ...,...,- Machine Learning (ML) is a powerful tool for...,- The research paper focuses on the challenges...,-

In [6]:
df['Palabras clave']

0     deep reinforcement learning (drl) model;fog-as...
1     cloud computing;network;traffic;ddos;packet an...
2     cloud computing;cybersecurity risks;ddos;machi...
3     ddos attack;cloud;deho;svm;kpca;databases;data...
4     anomaly detection;training;machine learning;pr...
5     aws cloud;intrusion detection;principal compon...
6     intrusion detection system;feature selection;m...
7     industrial internet of things;sensors;temperat...
8     distributed denial of service (ddos);cloud-bas...
9     distributed denial of service attacks;denial o...
10    cloud computing;anomaly detection;security;rev...
11    software-defined networks;ddos attack;deep rei...
12    architecture in the cloud for the detection dd...
13    internet of things (iot);cyber-security;distri...
14    intrusion detection;convolutional neural netwo...
15    network security system;improved restricted bo...
16    ddos attacks;cloud computing systems;entropy f...
17    ddos attacks;cloud computing;artificial ne

In [8]:
keywords = {} # dict[str, int]

for kws in df['Palabras clave'].str.split(';'):
    for kw in kws:
        if kw in keywords:
            keywords[kw] += 1
        else:
            keywords[kw] = 1

keywords

{'deep reinforcement learning (drl) model': 1,
 'fog-assisted cloud environment': 1,
 'software defined networking (sdn)': 1,
 'availability': 1,
 'legitimate packets': 1,
 'cloud computing': 14,
 'network': 1,
 'traffic': 1,
 'ddos': 6,
 'packet analyzer': 1,
 'trust management': 1,
 'cybersecurity risks': 1,
 'machine learning': 9,
 'bayscnn model': 1,
 'cloud detection': 1,
 'dimension reduction': 1,
 'ddos attack': 4,
 'cloud': 2,
 'deho': 1,
 'svm': 1,
 'kpca': 1,
 'databases': 1,
 'data samples': 1,
 'cyber security': 3,
 'anomaly detection': 5,
 'training': 1,
 'prediction algorithms': 1,
 'feature extraction': 4,
 'predictive models': 1,
 'security': 8,
 'network security': 6,
 'software defined network': 1,
 'internet of things': 3,
 'cloud network': 2,
 'aws cloud': 1,
 'intrusion detection': 3,
 'principal component analysis (pca)': 1,
 'fuzzy c-means': 1,
 'smart monkey optimization': 1,
 'autoencoder': 2,
 'cse-cic-ids-2018 dataset': 1,
 'intrusion detection system': 3,
 '

In [14]:
keywords = {'fog-assisted cloud environment': 1,
            'software defined networking (sdn)': 1,
            'availability': 1,
            'legitimate packets': 1,
            'network': 1,
            'traffic': 1,
            'ddos': 26,
            'packet analyzer': 1,
            'trust management': 1,
            'cybersecurity': 18,
            'machine learning': 12,
            'bayscnn model': 1,
            'dimension reduction': 1,
            'cloud': 28,
            'deho': 1,
            'svm': 1,
            'kpca': 1,
            'dataset': 6,
            'anomaly detection': 5,
            'training': 1,
            'prediction': 2,
            'feature extraction': 4,
            'network security': 6,
            'software defined network': 1,
            'iot': 8,
            'intrusion detection': 3,
            'principal component analysis (pca)': 1,
            'fuzzy c-means': 1,
            'smart monkey optimization': 1,
            'autoencoder': 2,
            'intrusion detection system': 3,
            'feature selection': 1,
            'deep learning': 16,
            'sensors': 1,
            'temperature sensors': 1,
            'computer crime': 2,
            'protocols': 1,
            'computer security': 1,
            'cybersecurity applications': 1,
            'federated learning': 1,
            'edge computing': 1,
            'network layer traffic analysis': 1,
            'transport layer traffic analysis': 1,
            'network traffic characterization': 1,
            'time series analysis': 1,
            'software-defined networks': 1,
            'metaheuristics': 5,
            'ensemble model': 2,
            'anomaly-based detection': 1,
            'rf classification algorithm': 1,
            'convolutional neural networks': 1,
            'ids': 2,
            'cnn': 2,
            'lstm': 1,
            'network security system': 1,
            'improved restricted boltzmann machine': 1,
            'granular-ball optimization algorithm': 1,
            'soft computing': 1,
            'entropy forecasting': 1,
            'genetic algorithm': 2,
            'artificial neural networks': 1,
            'extreme learning machine': 1,
            'differential evolution': 1,
            'data preprocessing': 1,
            'decision trees': 1,
            'support vector machine': 2,
            'naive bayes': 1,
            'k-nearest neighbors': 1,
            'attacks': 1,
            'mutual information': 1,
            'random forest': 1,
            'sdn': 2,
            'test-bed': 1,
            'classification': 1,
            'recurrent neural network': 2,
            'cic ids 2017': 1,
            'distributed denial of service': 2,
            'ensemble feature selection': 1,
            'long short-term memory (lstm)': 1,
            'feature engineering': 1,
            't-test': 1,
            'tree-based machine learning': 1,
            'dbscan': 1,
            'unsupervised learning': 1,
            'distributed computing': 1,
            'intrusion detection systems': 1,
            'attack mitigation': 1,
            'firewall': 1,
            'evolutionary algorithm': 1,
            'ensemble learning': 1,
            'botnet detection': 1,
            'reptile search algorithm': 1,
            'optimization algorithm': 1,
            'statistical features': 1,
            'hybrid algorithm (ha)': 1,
            'denial of service (dos)': 1,
            'platform as a service (paas)': 1,
            'infrastructure as a service (iaas)': 1,
            'software as a service (saas)': 1,
            'crow search algorithm': 1,
            'opposition based learning': 1,
            'brain': 1,
            'denial-of-service attack': 1,
            'particle swarm optimization (pso)': 1,
            'support vector machines': 1,
            'attack detection': 1,
            'cloud-computing': 2,
            'deep belief networks': 1,
            'denialof- service attacks': 1,
            'memory modeling': 1,
            'particle swarm': 1,
            'particle swarm optimization': 1,
            'services on demand': 1,
            'swarm optimization': 1,
            'long short-term memory': 1,
            'botnet': 2,
            'hidden markov models': 1,
            'learning algorithms': 2,
            'optimization': 2,
            'recurrent neural networks': 1,
            'botnet detections': 2,
            'botnets': 2,
            'classification algorithm': 1,
            'features extraction': 2,
            'hidden-markov models': 1,
            'risk assessment': 1,
            'optimisations': 1,
            'privacy': 1,
            'tuning': 1}

import re

texto = ' '.join([f'{k} ' * i for k, i in keywords.items()])
# Replace all '  ' to ' '
texto = re.sub(r'  ', ' ', texto).strip()
texto

'fog-assisted cloud environment software defined networking (sdn) availability legitimate packets network traffic ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos ddos packet analyzer trust management cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity cybersecurity machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning machine learning bayscnn model dimension reduction cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud cloud deho svm kpca dataset dataset dataset dataset dataset dataset ano